In [1]:
import numpy as np
import pandas as pd
import time
import commentsFilter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix, f1_score

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import sparse
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
stemmer = SnowballStemmer('english')
nltk.download('stopwords')
nltk.download('punkt')
%matplotlib inline

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Manu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Manu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# FILE_PATH = './Data/'
# SUBREDDIT = 'r_republican'
FILE_NAME = './Data/republican_democrats_raw.csv'

In [12]:
df = pd.read_csv(FILE_NAME, index_col=0)
df['Score'].astype(np.int32)
print(df.shape)
df.head()

(302932, 8)


,Author,Body,Score,Subreddit,Created,Id,Post,Post_id
0,5th_Law_of_Robotics,It would take him an hour to work his way thro...,1,democrats,1600191390,g5dngzh,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
1,Cupanudles,Hahahahahahahahahahahahahahahahahaha,1,democrats,1600191448,g5dnm3c,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
2,89SC,He couldn’t even make it through the SparkNote...,1,democrats,1600191720,g5do9pg,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
3,XAfricaSaltX,Suuuuuuuuuuuure,1,democrats,1600193029,g5drd0e,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
4,Donaldtrumpsmushroom,"If he reads with his bunghole, sure.",1,democrats,1600195657,g5dxn0n,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm


In [4]:
# df_cl = data[data['Score'] > 30]
# df_ng = data[data['Score'] < -10]
# df = pd.concat([df_cl, df_ng])
# df.shape

In [5]:
# converting subreddit vals to ints

index_democrat = df[df['Subreddit'] == 'democrats'].index
df['Subreddit'] =0 
df.loc[index_democrat, 'Subreddit'] =1

In [6]:
df[df['Subreddit']==1]

,Unnamed: 0,Author,Body,Score,Subreddit,Created,Id,Post,Post_id
0,0,5th_Law_of_Robotics,It would take him an hour to work his way thro...,1,1,1600191390,g5dngzh,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
1,1,Cupanudles,Hahahahahahahahahahahahahahahahahaha,1,1,1600191448,g5dnm3c,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
2,2,89SC,He couldn’t even make it through the SparkNote...,1,1,1600191720,g5do9pg,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
3,3,XAfricaSaltX,Suuuuuuuuuuuure,1,1,1600193029,g5drd0e,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
4,4,Donaldtrumpsmushroom,"If he reads with his bunghole, sure.",1,1,1600195657,g5dxn0n,Noted Bibliophobe Donald Trump Claims He Read ...,itdjfm
...,...,...,...,...,...,...,...,...,...
145551,145551,softwaredev,> it's ads you CHOSE to watch\n\nReally? peop...,2,1,1483677072,dc2bbs7,January Begins Roundtable,5lfpmc
145552,145552,NaN,Obama had charisma AND inspired a lot of peopl...,2,1,1483510589,dbz80jq,January Begins Roundtable,5lfpmc
145553,145553,NaN,> Really? people get to CHOOSE the ads nowaday...,1,1,1483690774,dc2i8vo,January Begins Roundtable,5lfpmc
145554,145554,softwaredev,> If you look at all her ads ...\n\nThe point ...,2,1,1483779412,dc42r8t,January Begins Roundtable,5lfpmc


In [7]:
# filtering 
# removing comments from automod, removed comments, comments with a neutral score 
# filterComments(df, filter_automod, filter_negative_comments, filter_deleted_posts, filter_neutral_comments):
commentsFilter.filterComments(df, 1, 0, 1, 0)
df.shape

Automod indexes: Int64Index([   114,    197,    748,    931,    946,   1100,   1101,   1158,
              1240,   1246,
            ...
            285696, 285771, 285811, 285871, 285898, 285962, 286035, 286060,
            286068, 288070],
           dtype='int64', length=7424)
Deleted/removed comments indexes: Int64Index([    27,     51,     52,     74,     75,     89,     93,     95,
               113,    158,
            ...
            302678, 302714, 302748, 302751, 302758, 302764, 302823, 302837,
            302852, 302889],
           dtype='int64', length=22198)


(273310, 9)

In [8]:
# # adding a column with negativ score label

# df['pos_score'] =0
# index_pos_score = df[df['Score'] >= 1].index
# df.loc[index_pos_score, 'pos_score'] =1
# df.head(3)

In [9]:
# coverting column to str (incase a comment contains only digits and is considered int)
df['Body'] = df['Body'].astype(str)
df['Body'].head(3)

0    It would take him an hour to work his way thro...
1                 Hahahahahahahahahahahahahahahahahaha
2    He couldn’t even make it through the SparkNote...
Name: Body, dtype: object

In [10]:
from sklearn.model_selection import train_test_split

# splitting data
x = df['Body', 'Score']
y = df['Subreddit']
xtr, xts, ytr, yts = train_test_split(x, y, test _size=0.30, stratify=y)


SyntaxError: invalid syntax (<ipython-input-10-340738f988c8>, line 6)

In [ ]:
# # # random undersampling for class with negative score for training set
# pos_class_count, neg_class_count = ytr.value_counts()
# xtrain = pd.concat([xtr, ytr], axis=1)

# xtrain.columns
# xtr_pos= xtrain[xtrain['pos_score'] ==1]
# xtr_neg = xtrain[xtrain['pos_score']==0]

# xtr_pos_under = xtr_pos.sample(neg_class_count)
# xtrain = pd.concat([xtr_pos_under, xtr_neg])
# xtr = xtrain['Body']
# ytr = xtrain['pos_score']
# print(f'xtr shape: {xtr.shape}  ytr shape: {ytr.shape}')

In [ ]:
"""pipeline params:
    text
    remove_stopwords --> [0,1] 
    enable_stemming --> [0, 1]
    n_gram   1-5 maybe? [0, 4]
    tokenizer  --> only nltk.word_tokenize for now
    tf_idf/ embeddi --> only tf_idf for now
"""

def nlp_pipeline(text, remove_stopwords, enable_stemming, n_gram, tokenizer ):
    pass 

In [ ]:
stop_words_list = stopwords.words('english')
wnl = WordNetLemmatizer()

def stemming_tokenizer(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [w for w in tokens if not w in stop_words_list]
    lems = [wnl.lemmatize(w) for w in filtered_tokens]
    #removing punctuatoin
    new_words= [word for word in lems if word.isalnum()]
    return new_words
    
def stem_tokens(tokens):
    return [stemmer.stem(w) for w in tokens]

def remove_stopwords(tokens):
    return [w for w in tokens if not w in stop_words_list]
    

    

In [ ]:
%%time


# converting comments to lowercase
xtr_comments = xtr.str.lower()
xts_comments = xts.str.lower()

tfidf = TfidfVectorizer(max_features=20000, tokenizer=stemming_tokenizer, ngram_range=(1, 1))


In [ ]:
%%time

# tfidf
xtr_vect = tfidf.fit_transform(xtr_comments)
xts_vect = tfidf.fit_transform(xts_comments)

In [ ]:
# horribl idea
from scipy import sparse
nRow_xtr = xtr_vect.shape[0]
nRow_xts = xts_vect.shape[0]

xtr_score_csr = sparse.csr_matrix(xtr['Score'].to_numpy()).reshape(nRow_xtr, 1)
xts_score_csr = sparse.csr_matrix(xts['Score'].to_numpy()).reshape(nRow_xts, 1)

xtr_sm = sparse.hstack([xtr_vect, xtr_score_csr])
xts_sm = sparse.hstack([xts_vect, xts_score_csr])

In [ ]:
# from scipy.sparse import coo_matrix, hstack
# bsbsbs = xtr['Score'].values
# bsbsbs.shape
# bs = sparse.hstack((xtr_vec, bsbsbs))

In [ ]:
# logistic regresssion
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

lr = LogisticRegression(max_iter=20000, class_weight={1:1})
lr.fit(xtr_sm, ytr)
pred = lr.predict(xts_sm)

print('\nConfusion matrix\n',confusion_matrix(yts, pred))
print(f'f1 score: {f1_score(yts, pred)}')


In [ ]:

# from sklearn.neural_network import MLPClassifier

# nn = MLPClassifier(random_state=1, max_iter=300)
# nn.fit(xtr_vect, ytr)
# preds = nn.predict(xts_vect)
# pred_probas= nn.predict_proba(xts_vect)

# print('\nConfusion matrix\n',confusion_matrix(yts, preds))
# print(f'f1 score: {f1_score(yts, pred)}')
